# Refresh KSATT Open Source Graph Database

### Usage

Neo4j Version 4.4.5 was used at time of updating this.

While all of these sources are relatively static, they do have version updates (on irregular cadences).  
It is not urgent to incorporate these updates, so running these updates should be done monthly or quarterly and require a more significant rebuild of the KSATT graph database as well.  

Prior to running this code for the first time, you should:
- Ensure you have all python packages installed on your local machine (and optionally in the virtual environment you use to run this).
- Create an empty new local Neo4j graph database. 
  - Start it upon creation, then stop it. 
  - Install the APOC plugin. 
  - Edit the settings file so that the minimum and maximum heap sizes are at least 1GB and 3GB respectively.
  - You'll need to restart the database after making those changes to the configuration.
- Substitute in your file path to the `neo4j_import_folder` variable in the *Prep* codeblock under *Download Data*. 
  - This file path can be found through Neo4j Desktop (... > Open folder > DBMS > import). I will open this directory in Finder then select "New Terminal at Folder" and `pwd` to copy and paste the full path. 
- Manually copy the `opm_onet_crosswalk.csv` file from this repository into the Neo4j DBMS import folder. 

Make sure your Neo4j database is started before running this code.  
When this code is ran, it:
- Downloads data from ONET, ESCO and NICE in the form of CSVs into the Neo4j import folder. **Remember to change `download_new_data` flag to True.**
- Cleans all data in the Neo4j import folder in preparation for upload. 
- Clears a local **existing and running** Neo4j graph database, then uploads the newly downloaded and cleaned data. 

### To Do
- Incorporate [ONET Crosswalks](https://www.onetcenter.org/crosswalks.html).
- Possibly incorporate USA Jobs. 

### Cahnges in code to run in MemGraph
- apoc.periodic.iterate -> periodic.iterate
- WITH HEADERS -> WITH HEADER and move after csv file location
- batchSize -> batch_size and only accepts one parameter, number in batch
- YIELD operation did not work, changed to YIELD * RETURN * for now
- added some functions specifically for mage and constraints

# Upload to MEM

## Clear Database and Assert Schema

In [ ]:
# MATCH (n) DETACH DELETE n 
#from neo4j import GraphDatabase
 
#URI = "bolt://localhost:7687"
#AUTH = ("  "")
 
#with GraphDatabase.driver(URI, auth=AUTH) as client:

#    query = "CREATE CONSTRAINT ON (n:ONET_Major_Group) ASSERT n.key IS UNIQUE;"
#   client.execute_query(query)
 
#with GraphDatabase.driver(URI, auth=AUTH).session() as session:
#    session.run("CREATE CONSTRAINT ON (n:ONET_Major_Group) ASSERT n.key IS UNIQUE;")

## Constraints and Indexes

In [ ]:
print("Removing all nodes and clearing the database schema for a fresh start")
# NOTE: had issue with broken connection or wire error, something like that - testing to see if upping heap size in neo4j.conf for the new database helps
 MATCH (n) DETACH DELETE n 
# CALL apoc.schema.assert({},{}, true)  # clear schema

print("Assert unique index properties for nodes related to employee data - this makes searching faster and organization neater")
run_constraint_query("CREATE CONSTRAINT ON (n:ONET_Occupation) ASSERT n.key, n.title, n.description IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ONET_Occupation(key);")
run_constraint_query("CREATE CONSTRAINT ON (n:ONET_Major_Group) ASSERT n.key IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ONET_Major_Group(key);")
run_constraint_query("CREATE CONSTRAINT ON (n:ONET_Minor_Group) ASSERT n.key IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ONET_Minor_Group(key);")
run_constraint_query("CREATE CONSTRAINT ON (n:ONET_Broad_Occupation_Group) ASSERT n.key IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ONET_Broad_Occupation_Group(key);")
run_constraint_query("CREATE CONSTRAINT ON (n:ONET_Detailed_Occupation_Group) ASSERT n.key IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ONET_Detailed_Occupation_Group(key);")

run_constraint_query("CREATE CONSTRAINT ON (n:ONET_Scale) ASSERT n.key IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ONET_Scale(key);")
run_constraint_query("CREATE CONSTRAINT ON (n:ONET_Segment) ASSERT n.key, n.title IS UNIQUE;") 
run_constraint_query("CREATE INDEX ON :ONET_Segment(key);")
run_constraint_query("CREATE CONSTRAINT ON (n:ONET_Family) ASSERT n.key, n.title IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ONET_Family(key);")
run_constraint_query("CREATE CONSTRAINT ON (n:ONET_Class) ASSERT n.key, n.title IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ONET_Class(key);")
run_constraint_query("CREATE CONSTRAINT ON (n:ONET_Commodity) ASSERT n.key, n.title IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ONET_Commodity(key);")

run_constraint_query("CREATE CONSTRAINT ON (n:ONET_Technology_Skills) ASSERT n.key, n.title IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ONET_Technology_Skills(key);")
run_constraint_query("CREATE CONSTRAINT ON (n:ONET_Tools) ASSERT n.key, n.title IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ONET_Tools(key);")
run_constraint_query("CREATE CONSTRAINT ON (n:ONET_Technology_Skills_Example) ASSERT n.title IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :NET_Technology_Skills_Example(title);")
run_constraint_query("CREATE CONSTRAINT ON (n:ONET_Tools_Example) ASSERT n.title IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ONET_Tools_Example(title);")
run_constraint_query("CREATE CONSTRAINT ON (n:xc) ASSERT n.key, n.title IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ONET_Tools_Example(title);")

run_constraint_query("CREATE CONSTRAINT ON (n:ONET_Abilities) ASSERT n.key, n.title, n.description IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ONET_Abilities(key);")
run_constraint_query("CREATE CONSTRAINT ON (n:ONET_Interests_And_Work_Values) ASSERT n.key, n.title IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ONET_Interests_And_Work_Values(key);")
run_constraint_query("CREATE CONSTRAINT ON (n:ONET_Work_Styles) ASSERT n.key, n.title, n.description IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ONET_Work_Styles(key);")
run_constraint_query("CREATE CONSTRAINT ON (n:ONET_Basic_Skills) ASSERT n.key, n.title, n.description IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ONET_Basic_Skills(key);")
run_constraint_query("CREATE CONSTRAINT ON (n:ONET_Cross_Functional_Skills) ASSERT n.key, n.title, n.description IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ONET_Cross_Functional_Skills(key);")
run_constraint_query("CREATE CONSTRAINT ON (n:ONET_Knowledge) ASSERT n.key, n.title, n.description IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ONET_Knowledge(key);")
run_constraint_query("CREATE CONSTRAINT ON (n:ONET_Education) ASSERT n.key, n.title, n.description IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ONET_Education(key);")
run_constraint_query("CREATE CONSTRAINT ON (n:ONET_Education_Category) ASSERT n.key IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ONET_Education_Category(key);")
run_constraint_query("CREATE CONSTRAINT ON (n:ONET_Experience_And_Training) ASSERT n.key, n.title, n.description IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ONET_Experience_And_Training(key);")
run_constraint_query("CREATE CONSTRAINT ON (n:ONET_Experience_And_Training_Category) ASSERT n.key IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ONET_Experience_And_Training_Category(key);")
run_constraint_query("CREATE CONSTRAINT ON (n:ONET_Generalized_Work_Activities) ASSERT n.key, n.title, n.description IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ONET_Generalized_Work_Activities(key);")
run_constraint_query("CREATE CONSTRAINT ON (n:ONET_Intermediate_Work_Activities) ASSERT n.key, n.title, n.description IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ONET_Intermediate_Work_Activities(key);")
run_constraint_query("CREATE CONSTRAINT ON (n:ONET_Detailed_Work_Activities) ASSERT n.key, n.title, n.description IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ONET_Detailed_Work_Activities(key);")
run_constraint_query("CREATE CONSTRAINT ON (n:ONET_Work_Context) ASSERT n.key, n.title, n.description IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ONET_Work_Context(key);")
  
run_constraint_query("CREATE CONSTRAINT ON (n:ESCO_Skills) ASSERT n.uri, n.key, n.alt_labels IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ESCO_Skills(key);")
run_constraint_query("CREATE CONSTRAINT ON (n:ESCO_Knowledge) ASSERT n.uri, n.key, n.description, n.alt_labels IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ESCO_Knowledge(key);")
run_constraint_query("CREATE CONSTRAINT ON (n:ESCO_Language) ASSERT n.uri, n.key, n.title, n.description, n.alt_labels IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ESCO_Language(key);")
run_constraint_query("CREATE CONSTRAINT ON (n:ESCO_Attitudes_Values) ASSERT n.uri, n.key, n.description, n.alt_labels IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ESCO_Attitudes_Values(uri);")
run_constraint_query("CREATE CONSTRAINT ON (n:ESCO_Occupation_Group) ASSERT n.uri, n.key, n.alt_labels, n.description IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ESCO_Occupation_Group(uri);")
run_constraint_query("CREATE CONSTRAINT ON (n:ESCO_Occupation) ASSERT n.uri, n.title, n.alt_labels, n.description, n.key IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :ESCO_Occupation(uri);")

run_constraint_query("CREATE CONSTRAINT ON (n:NICE_Category) ASSERT n.title, n.acronym, n.description IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :NICE_Category(title);")
run_constraint_query("CREATE CONSTRAINT ON (n:NICE_Area) ASSERT n.title, n.acronym, n.description IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :NICE_Area(title);")
run_constraint_query("CREATE CONSTRAINT ON (n:NICE_Workrole) ASSERT n.key, n.title, n.description IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :NICE_Workrole(key);")
run_constraint_query("CREATE CONSTRAINT ON (n:OPM_Cybersecurity_Category) ASSERT n.key IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :OPM_Cybersecurity_Category(key);")
run_constraint_query("CREATE CONSTRAINT ON (n:NICE_Knowledge) ASSERT n.title IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :NICE_Knowledge(title);")
run_constraint_query("CREATE CONSTRAINT ON (n:NICE_Skills) ASSERT n.title IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :NICE_Skills(title);")
run_constraint_query("CREATE CONSTRAINT ON (n:NICE_Abilities) ASSERT n.title IS UNIQUE;")
run_constraint_query("CREATE INDEX ON :NICE_Abilities(title);")
run_constraint_query("CREATE CONSTRAINT ON (n:NICE_Tasks) ASSERT n.title IS UNIQUE;") 
run_constraint_query("CREATE INDEX ON :NICE_Tasks(title);")


## Upload ONET Data

In [ ]:
//print("Create a hierarchical pattern of Segment, Family, Class and Commodity nodes")

// Hierarchy created from Segment <- Family <- Class <- Commodity
 
LOAD CSV FROM  '/usr/lib/memgraph/import/onet/trusted_unspscreference.csv' WITH HEADER AS row RETURN row 
MERGE (a:ONET_Segment {key: row.segment_code})
SET a.title = toLower(row.segment_title)
MERGE (b:ONET_Family {key: row.family_code})
SET b.title = toLower(row.family_title)
MERGE (c:ONET_Class {key: row.class_code})
SET c.title = toLower(row.class_title)
MERGE (d:ONET_Commodity {key: row.commodity_code})
MERGE (a)<-[:SUB_SEGMENT_OF]-(b)
MERGE (b)<-[:SUB_SEGMENT_OF]-(c)
MERGE (c)<-[:SUB_SEGMENT_OF]-(d)
 

//print("Check that the Commodity hierachy and Technology Skills/Tools queries completed as expected")
//check_queries = []
MATCH p=(:ONET_Technology_Skills)--(:ONET_Technology_Skills_Example) RETURN count(p) AS num 
MATCH p=(:ONET_Tools)--(:ONET_Tools_Example) RETURN count(p) AS num 
MATCH (a) WHERE a:ONET_Commodity AND a:ONET_Technology_Skills RETURN count(a) AS num 
MATCH (a) WHERE a:ONET_Commodity AND a:ONET_Tools RETURN count(a) AS num 
MATCH p=(:ONET_Segment)<-[:SUB_SEGMENT_OF]-(:ONET_Family)<-[:SUB_SEGMENT_OF]-(:ONET_Class)<-[:SUB_SEGMENT_OF]-(:ONET_Commodity) RETURN count(p) AS num 
MATCH p=(:ONET_Segment)<-[:SUB_SEGMENT_OF]-(:ONET_Family)<-[:SUB_SEGMENT_OF]-(:ONET_Class)<-[:SUB_SEGMENT_OF]-(:ONET_Technology_Skills) RETURN count(p) AS num 
MATCH p=(:ONET_Segment)<-[:SUB_SEGMENT_OF]-(:ONET_Family)<-[:SUB_SEGMENT_OF]-(:ONET_Class)<-[:SUB_SEGMENT_OF]-(:ONET_Tools) RETURN count(p) AS num 
MATCH p=(a:ONET_Occupation_Specific_Information {key:'5.F'})<-[:SUB_ELEMENT_OF]-(b:ONET_Technology_Skills) RETURN count(p) AS num 
MATCH p=(a:ONET_Occupation_Specific_Information {key:'5.G'})<-[:SUB_ELEMENT_OF]-(b:ONET_Tools) RETURN count(p) AS num 
# these should really be more than 1
for cq in check_queries:
    results = run_reg_query(cq)['num'][0]
    if results <= 1:
        interrupt("Segment/Family/Class/Commodity/Tech Skills/Tools  "there should be more nodes and relationships")

num_tech =  MATCH (a:ONET_Technology_Skills) RETURN count(a) AS num ['num'][0]
num_tools =  MATCH (a:ONET_Tools) RETURN count(a) AS num ['num'][0]
num_comm =  MATCH (a:ONET_Commodity) RETURN count(a) AS num ['num'][0]
if num_comm != (num_tech+num_tools-2):
    interrupt("Number of Commodity vs. Tech Skills & Tools  "there should be an even number of Commodities and Tech Skills + Tools (-2 because of ONET taxonomy import)")

In [ ]:
print("Create a hierarchy of work activities (generalized, intermediate, detailed)")
# Use: be able to look at work activities at different levels of granularity
# **ONET_Generalized_Work_Activities** were already created from the contentmodelreference.csv (all **ONET_Element** nodes that started as 4.A were re-labeled as **ONET_Generalized_Work_Activities**)
# Intermediate & Detailed were not detailed in contentmodelreference.csv but are in this document, so we're creating these
# **ONET_Generalized_Work_Activities** <- **ONET_Intermediate_Work_Activities** <- **ONET_Detailed_Work_Activities**
# **ONET_Intermediate_Work_Activities** and **ONET_Detailed_Work_Activities** are related to their parents in the element hierarchy

# Generalized_Work_Activities were already created from contentmodelreference.csv, so we need to match those
 
 LOAD CSV FROM  '/usr/lib/memgraph/import/onet/trusted_dwareference.csv' WITH HEADER AS row RETURN row 
"
MATCH (a:ONET_Generalized_Work_Activities {key: row.element_id})
MATCH (bb:ONET_Occupational_Requirements {title:'intermediate work activities'})
MATCH (cc:ONET_Occupational_Requirements {title:'detailed work activities'})
MERGE (b:ONET_Intermediate_Work_Activities {key: row.iwa_id, title: toLower(row.iwa_title)})
MERGE (c:ONET_Detailed_Work_Activities {key: row.dwa_id, title: toLower(row.dwa_title)})

MERGE (a)<-[:DETAIL_OF]-(b)
MERGE (b)<-[:DETAIL_OF]-(c)
MERGE (bb)<-[:SUB_ELEMENT_OF]-(b)
MERGE (cc)<-[:SUB_ELEMENT_OF]-(c)

 {batch_size:1000}) YIELD * RETURN *; 

check_queries = []

MATCH p=(:ONET_Occupational_Requirements)<-[:SUB_ELEMENT_OF]-(:ONET_Generalized_Work_Activities) RETURN count(p) AS num 
MATCH p=(:ONET_Occupational_Requirements)<-[:SUB_ELEMENT_OF]-(:ONET_Intermediate_Work_Activities) RETURN count(p) AS num 
MATCH p=(:ONET_Occupational_Requirements)<-[:SUB_ELEMENT_OF]-(:ONET_Detailed_Work_Activities) RETURN count(p) AS num 
MATCH p=(:ONET_Generalized_Work_Activities)<-[:DETAIL_OF]-(:ONET_Intermediate_Work_Activities)<-[:DETAIL_OF]-(:ONET_Detailed_Work_Activities) RETURN count(p) AS num 

for cq in check_queries:
    results = run_reg_query(cq)['num'][0]
    if results < 1:
        interrupt("Work Activities  "there should be chains of work activities")

In [ ]:
print("Create task nodes")
# Use: different tasks performed as part of occupations or work activities imply skills, which we are interested in
# These were not included in contentmodelreference.csv, only the parent Tasks **ONET_Element** node was, so this loads detailed **ONET_Tasks** nodes
 
 LOAD CSV FROM  '/usr/lib/memgraph/import/onet/trusted_taskstatements.csv' WITH HEADER AS row RETURN row 
"
MATCH (a:ONET_Occupation_Specific_Information {title: 'tasks'})
MERGE (b:ONET_Tasks {title: toLower(row.task)})
ON CREATE SET b.key = row.task_id, b.description = toLower(row.task), b.type = toLower(row.task_type)
ON MATCH SET b.key = row.task_id+', '+b.key
MERGE (a)<-[:SUB_ELEMENT_OF]-(b)
 {batch_size:10000}) YIELD * RETURN *; 
MATCH p=(:ONET_Occupation_Specific_Information {title: 'tasks'})<-[:SUB_ELEMENT_OF]-(:ONET_Tasks) RETURN count(p) AS num ['num'][0]
if results <= 1:
    interrupt("Tasks  "there should be more relationships")

print("Match tasks to detailed work activities")
# Because **ONET_Tasks** are required in order to complete work activities, we want to associate **ONET_Detailed_Work_Activities** with requisite **ONET_Tasks**.
# We don't create any nodes, we just MATCH existing **ONET_Tasks** and **ONET_Detailed_Work_Activities** nodes.
 
 LOAD CSV FROM  '/usr/lib/memgraph/import/onet/trusted_taskstodwas.csv' WITH HEADER AS row RETURN row 
"
MATCH (a:ONET_Detailed_Work_Activities {key: row.dwa_id})
MATCH (b:ONET_Tasks {title: toLower(row.task)})
WITH a, b, row
MERGE (a)<-[:TASK_FOR_DWA]-(b)
 {batch_size:10000}) YIELD * RETURN *; 
MATCH p=(:ONET_Detailed_Work_Activities)<-[:TASK_FOR_DWA]-(:ONET_Tasks) RETURN count(p) AS num ['num'][0]
if results <= 1:
    interrupt('Tasks & DWAs', 'there should be relationships between DWAs and Tasks')

In [ ]:
print("Enrich the existing education and experiences and training data by adding categories")

 
 LOAD CSV FROM  '/usr/lib/memgraph/import/onet/trusted_educationtrainingandexperiencecategories.csv' WITH HEADER AS row RETURN row 
"
MATCH (a:ONET_Education {key: row.element_id})
MERGE (b:ONET_Education_Category {key: row.scale_id+'.'+row.category, title: toLower(row.category_description)})
MERGE (a)<-[:CATEGORY_OF]-(b)
 {batch_size:10000}) YIELD * RETURN *; 

 
 LOAD CSV FROM  '/usr/lib/memgraph/import/onet/trusted_educationtrainingandexperiencecategories.csv' WITH HEADER AS row RETURN row 
"
MATCH (a:ONET_Experience_And_Training {key: row.element_id})
MERGE (b:ONET_Experience_And_Training_Category {key: row.scale_id+'.'+row.category, title: toLower(row.category_description)})
MERGE (a)<-[:CATEGORY_OF]-(b)
 {batch_size:10000}) YIELD * RETURN *; 


MATCH p=(:ONET_Education)<-[:CATEGORY_OF]-(:ONET_Education_Category) RETURN count(p) AS num ['num'][0]
if results <= 1:
    interrupt("Education Categories  "there should be relationships between Education and categories")
MATCH p=(:ONET_Experience_And_Training)<-[:CATEGORY_OF]-(:ONET_Experience_And_Training_Category) RETURN count(p) AS num ['num'][0]
if results <= 1:
    interrupt("Experience and Training Categories  "there should be relationships between Experience and Training and categories")

MATCH p=(:ONET_Education)<-[:CATEGORY_OF]-(:ONET_Education_Category)-[:CATEGORY_OF]->(:ONET_Education) RETURN count(p) AS num ['num'][0]
if results != 0:
    interrupt("Education Categories  "there should not be relationships between an Education category and multiple Education nodes")
MATCH p=(:ONET_Experience_And_Training)<-[:CATEGORY_OF]-(:ONET_Experience_And_Training_Category)-[:CATEGORY_OF]->(:ONET_Experience_And_Training) RETURN count(p) AS num ['num'][0]
if results != 0:
    interrupt("Experience and Training Categories  "there should not be relationships between an Experience and Training category and multiple Experience and Training nodes")

In [ ]:
print("Create relationships between occupations and the various KSATTs derived from ONET")
# 1.  All KSATT nodes were originally **ONET_Element** nodes, they were just re-labeled. So they all have an id and title, and _SUB_ELEMENT_OF_ relationship. These are not being created, only matched.
# 2.  Now they all will have relationships with **ONET_Occupation** nodes - these are _FOUND_IN_ relationships at the core, but are named differently based on the type of scale that the relationship indicates (e.g. _LV_FOUND_IN_ for level, _IM_FOUND_IN_ for importance, etc.). These will be cleaned up later.

 
 LOAD CS FROM  '/usr/lib/memgraph/import/onet/trusted_abilities.csv'V WITH HEADER AS row RETURN row 
"
MATCH (a:ONET_Occupation {key: row.onet_soc_code})
MATCH (b:ONET_Abilities {key: row.element_id})
WITH a, b, row
CALL apoc.merge.relationship(b, toString(row.scale_id)+'_FOUND_IN', {data_value: toFloat(row.data_value)}, {}, a, {}) YIELD rel RETURN rel
 {batch_size:10000}) YIELD * RETURN *; 

 
 LOAD CSV FROM  '/usr/lib/memgraph/import/onet/trusted_interests.csv' WITH HEADER AS row RETURN row 
"
MATCH (a:ONET_Occupation {key: row.onet_soc_code})
MATCH (b:ONET_Interests_And_Work_Values {key: row.element_id})
WITH a, b, row
CALL apoc.merge.relationship(b, toString(row.scale_id)+'_FOUND_IN', {data_value: toFloat(row.data_value)}, {}, a, {}) YIELD rel RETURN rel
 {batch_size:1000}) YIELD * RETURN *; 


 
 LOAD CSV FROM  '/usr/lib/memgraph/import/onet/trusted_workstyles.csv' WITH HEADER AS row RETURN row 
"
MATCH (a:ONET_Occupation {key: row.onet_soc_code})
MATCH (b:ONET_Work_Styles {key: row.element_id})
WITH a, b, row
CALL apoc.merge.relationship(b, toString(row.scale_id)+'_FOUND_IN', {data_value: toFloat(row.data_value)}, {}, a, {}) YIELD rel RETURN rel
 {batch_size:1000}) YIELD * RETURN *; 

 
 LOAD CSV FROM  '/usr/lib/memgraph/import/onet/trusted_skills.csv' WITH HEADER AS row RETURN row 
"
MATCH (a:ONET_Occupation {key: row.onet_soc_code})
MATCH (b:ONET_Basic_Skills {key: row.element_id})
WITH a, b, row
CALL apoc.merge.relationship(b, toString(row.scale_id)+'_FOUND_IN', {data_value: toFloat(row.data_value)}, {}, a, {}) YIELD rel RETURN rel
 {batch_size:10000}) YIELD * RETURN *; 

 
 LOAD CSV FROM  '/usr/lib/memgraph/import/onet/trusted_skills.csv' WITH HEADER AS row RETURN row 
"
MATCH (a:ONET_Occupation {key: row.onet_soc_code})
MATCH (b:ONET_Cross_Functional_Skills {key: row.element_id})
WITH a, b, row
CALL apoc.merge.relationship(b, toString(row.scale_id)+'_FOUND_IN', {data_value: toFloat(row.data_value)}, {}, a, {}) YIELD rel RETURN rel
 {batch_size:10000}) YIELD * RETURN *; 

 
 LOAD CSV FROM  '/usr/lib/memgraph/import/onet/trusted_knowledge.csv' WITH HEADER AS row RETURN row 
"
MATCH (a:ONET_Occupation {key: row.onet_soc_code})
MATCH (b:ONET_Knowledge {key: row.element_id})
WITH a, b, row
CALL apoc.merge.relationship(b, toString(row.scale_id)+'_FOUND_IN', {data_value: toFloat(row.data_value)}, {}, a, {}) YIELD rel RETURN rel
 {batch_size:10000}) YIELD * RETURN *; 

 
 LOAD CSV FROM  '/usr/lib/memgraph/import/onet/trusted_educationtrainingandexperience.csv' WITH HEADER AS row RETURN row 
"
MATCH (a:ONET_Occupation {key: row.onet_soc_code})
MATCH (b:ONET_Education_Category {key: row.scale_id+'.'+row.category})
WITH a, b, row
CALL apoc.merge.relationship(b, toString(row.scale_id)+'_FOUND_IN', {data_value: toFloat(row.data_value)}, {}, a, {}) YIELD rel RETURN rel
 {batch_size:10000}) YIELD * RETURN *; 

 
 LOAD CSV FROM '/usr/lib/memgraph/import/onet/trusted_educationtrainingandexperience.csv' WITH HEADER AS row RETURN row 
"
MATCH (a:ONET_Occupation {key: row.onet_soc_code})
MATCH (b:ONET_Experience_And_Training_Category {key: row.scale_id+'.'+row.category})
WITH a, b, row
CALL apoc.merge.relationship(b, toString(row.scale_id)+'_FOUND_IN', {data_value: toFloat(row.data_value)}, {}, a, {}) YIELD rel RETURN rel
 {batch_size:10000}) YIELD * RETURN *; 

# here the GWAs are being matched but really we associated the DWA with the Occupations
 
 LOAD CSV FROM  '/usr/lib/memgraph/import/onet/trusted_workactivities.csv' WITH HEADER AS row RETURN row 
"
MATCH (a:ONET_Occupation {key: row.onet_soc_code})
MATCH (b:ONET_Generalized_Work_Activities {key: row.element_id})
WITH a, b, row
CALL apoc.merge.relationship(b, toString(row.scale_id)+'_FOUND_IN', {data_value: toFloat(row.data_value)}, {}, a, {}) YIELD rel RETURN rel
 {batch_size:10000}) YIELD * RETURN *; 

 
 LOAD CSV FROM  '/usr/lib/memgraph/import/onet/trusted_workcontext.csv' WITH HEADER AS row RETURN row 
"
MATCH (a:ONET_Occupation {key: row.onet_soc_code})
MATCH (b:ONET_Work_Context {key: row.element_id})
WITH a, b, row
CALL apoc.merge.relationship(b, toString(row.scale_id)+'_FOUND_IN', {data_value: toFloat(row.data_value)}, {}, a, {}) YIELD rel RETURN rel
 {batch_size:1000}) YIELD * RETURN *; 

 
 LOAD CSV FROM  '/usr/lib/memgraph/import/onet/trusted_taskratings.csv' WITH HEADER AS row RETURN row 
"
MATCH (a:ONET_Occupation {key: row.onet_soc_code})
MATCH (b:ONET_Tasks {key: row.task_id})
WITH a, b, row
CALL apoc.merge.relationship(b, toString(row.scale_id)+'_FOUND_IN', {data_value: toFloat(row.data_value)}, {}, a, {}) YIELD rel RETURN rel
 {batch_size:10000}) YIELD * RETURN *; 

# TODO: no weights 
 
 LOAD CSV FROM  '/usr/lib/memgraph/import/onet/trusted_technologyskills.csv' WITH HEADER AS row RETURN row 
"
MATCH (a:ONET_Occupation {key: row.onet_soc_code})
MATCH (b:ONET_Technology_Skills {key: row.commodity_code})
WITH a, b, row
MERGE (b)-[:FOUND_IN]->(a)
 {batch_size:10000}) YIELD * RETURN *; 

# TODO: no weights 
 
 LOAD CSV FROM  '/usr/lib/memgraph/import/onet/trusted_toolsused.csv' WITH HEADER AS row RETURN row 
"
MATCH (a:ONET_Occupation {key: row.onet_soc_code})
MATCH (b:ONET_Tools {key: row.commodity_code})
WITH a, b, row
MERGE (b)-[:FOUND_IN]->(a)
 {batch_size:10000}) YIELD * RETURN *; 

check_queries = []

MATCH p=(:ONET_Occupation)<--(:ONET_Abilities) RETURN count(p) AS num 
MATCH p=(:ONET_Occupation)<--(:ONET_Interests_And_Work_Values) RETURN count(p) AS num 
MATCH p=(:ONET_Occupation)<--(:ONET_Work_Styles) RETURN count(p) AS num 
MATCH p=(:ONET_Occupation)<--(:ONET_Basic_Skills) RETURN count(p) AS num 
MATCH p=(:ONET_Occupation)<--(:ONET_Cross_Functional_Skills) RETURN count(p) AS num 
MATCH p=(:ONET_Occupation)<--(:ONET_Education_Category) RETURN count(p) AS num 
MATCH p=(:ONET_Occupation)<--(:ONET_Experience_And_Training_Category) RETURN count(p) AS num 
MATCH p=(:ONET_Occupation)<--(:ONET_Generalized_Work_Activities) RETURN count(p) AS num 
MATCH p=(:ONET_Occupation)<--(:ONET_Work_Context) RETURN count(p) AS num 
MATCH p=(:ONET_Occupation)<--(:ONET_Tasks) RETURN count(p) AS num 
MATCH p=(:ONET_Occupation)<--(:ONET_Technology_Skills) RETURN count(p) AS num 
MATCH p=(:ONET_Occupation)<--(:ONET_Tools) RETURN count(p) AS num 

for cq in check_queries:
    results = run_reg_query(cq)['num'][0]
    if results <= 0:
        interrupt(cq, "there should be relationships")

In [ ]:
print("Clean FOUND_IN relationships")
# gather all the different types of FOUND_IN relationships
relationships = [x['rel'] for x in graph.run("""MATCH (a)-[r]-(b) WHERE type(r) CONTAINS 'FOUND_IN' RETURN DISTINCT type(r) AS rel .data()]
# convert all the different types of FOUND_IN relationships to a FOUND_IN relationship with a normalized property of it's type instead
# e.g. IM Scale from 1 to 10 | IM_FOUND_IN {data_value: 7} -> FOUND_IN {IM: 0.7}
for r in relationships:
    if r != 'FOUND_IN':
        prefix = r.replace('_FOUND_IN', '')
        print(prefix)
        graph.run("""
            CALL {{MATCH (a:ONET_Scale {{key: '{}'}}) RETURN a.max AS max LIMIT 1}}
            WITH max
            MATCH (a)<-[r1:{}_FOUND_IN]-(b) MERGE (a)<-[r2:FOUND_IN]-(b) SET r2.{} = toFloat(r1.data_value)/max
 "".format(prefix, prefix, prefix))
        graph.run("""MATCH (a)<-[r1:{}_FOUND_IN]-(b) DELETE r1""".format(prefix))
check_queries = []
MATCH p=(:ONET_Occupation)<-[:FOUND_IN]-(:ONET_Abilities) RETURN count(p) AS num 
MATCH p=(:ONET_Occupation)<-[:FOUND_IN]-(:ONET_Interests_And_Work_Values) RETURN count(p) AS num 
MATCH p=(:ONET_Occupation)<-[:FOUND_IN]-(:ONET_Work_Styles) RETURN count(p) AS num 
MATCH p=(:ONET_Occupation)<-[:FOUND_IN]-(:ONET_Basic_Skills) RETURN count(p) AS num 
MATCH p=(:ONET_Occupation)<-[:FOUND_IN]-(:ONET_Cross_Functional_Skills) RETURN count(p) AS num 
MATCH p=(:ONET_Occupation)<-[:FOUND_IN]-(:ONET_Education_Category) RETURN count(p) AS num 
MATCH p=(:ONET_Occupation)<-[:FOUND_IN]-(:ONET_Experience_And_Training_Category) RETURN count(p) AS num 
MATCH p=(:ONET_Occupation)<-[:FOUND_IN]-(:ONET_Generalized_Work_Activities) RETURN count(p) AS num 
MATCH p=(:ONET_Occupation)<-[:FOUND_IN]-(:ONET_Work_Context) RETURN count(p) AS num 
MATCH p=(:ONET_Occupation)<-[:FOUND_IN]-(:ONET_Tasks) RETURN count(p) AS num 
MATCH p=(:ONET_Occupation)<-[:FOUND_IN]-(:ONET_Technology_Skills) RETURN count(p) AS num 
MATCH p=(:ONET_Occupation)<-[:FOUND_IN]-(:ONET_Tools) RETURN count(p) AS num 
for cq in check_queries:
    results = run_reg_query(cq)['num'][0]
    if results <= 0:
        interrupt("Relationships between ONET Occupations and KSATTs  "there should be relationships")
results = [item['rel'] for item in graph.run("""MATCH (a)-[r]-(b) WHERE type(r) CONTAINS 'FOUND_IN' RETURN DISTINCT type(r) AS rel .data()]
print(results)
if len(results) > 1 or 'FOUND_IN' not in results:
    interrupt("FOUND_IN relationship cleaning  "only FOUND_IN relationship left should be not prefixxed by anything")

## Upload ESCO Data
Notes:
concept_Schemes didn't have anything helpful.  
Everything from ictSkills was in transversalSkillsCollection.

In [ ]:
print("Add skill, knowledge, language, and attitudes and value hierarchy")

# add skill hierarchy 
 
 LOAD CSV FROM  '/usr/lib/memgraph/import/esco/trusted_skillshierarchy_en.csv' WITH HEADER AS row RETURN row 
"
WHERE row.level_3_code <> '' AND row.level_0_preferred_term = 'skills'
MERGE (a:ESCO_Skills {uri: row.level_1_uri})
ON CREATE SET a.title = row.level_1_preferred_term
MERGE (b:ESCO_Skills {uri: row.level_2_uri})
ON CREATE SET b.title = row.level_2_preferred_term
MERGE (c:ESCO_Skills {uri: row.level_3_uri})
ON CREATE SET c.title = row.level_3_preferred_term, c.description = row.description

MERGE (a)<-[:SUB_ELEMENT_OF]-(b)
MERGE (b)<-[:SUB_ELEMENT_OF]-(c)
 
{batch_size:1000}) YIELD * RETURN *; 

# add knowledge hierarchy
 
 LOAD CSV FROM  '/usr/lib/memgraph/import/esco/trusted_skillshierarchy_en.csv' WITH HEADER AS row RETURN row 
"
WHERE row.level_3_preferred_term <> '' AND row.level_0_preferred_term = 'knowledge'
MERGE (a:ESCO_Knowledge {uri: row.level_1_uri})
ON CREATE SET a.title = row.level_1_preferred_term
MERGE (b:ESCO_Knowledge {uri: row.level_2_uri})
ON CREATE SET b.title = row.level_2_preferred_term
MERGE (c:ESCO_Knowledge {uri: row.level_3_uri})
ON CREATE SET c.title = row.level_3_preferred_term

MERGE (a)<-[:SUB_ELEMENT_OF]-(b)
MERGE (b)<-[:SUB_ELEMENT_OF]-(c)

WITH c, row
WHERE row.description <> ''
SET c.description = row.description
 
{batch_size:1000}) YIELD * RETURN *; 

# add language hierarchy
 
 LOAD CSV FROM  '/usr/lib/memgraph/import/esco/trusted_skillshierarchy_en.csv' WITH HEADER AS row RETURN row 
"
WHERE row.level_1_code <> '' AND row.level_0_preferred_term = 'language skills and knowledge'
MERGE (a:ESCO_Language {uri: row.level_1_uri})
ON CREATE SET a.title = row.level_1_preferred_term, a.description = row.description
 
{batch_size:1000}) YIELD * RETURN *; 

# add attitudes and values hierarchy 
 
 LOAD CSV FROM  '/usr/lib/memgraph/import/esco/trusted_skillshierarchy_en.csv' WITH HEADER AS row RETURN row 
"
WHERE row.level_3_code <> '' AND row.level_0_preferred_term = 'attitudes and values'
MERGE (a:ESCO_Attitudes_Values {uri: row.level_1_uri})
ON CREATE SET a.title = row.level_1_preferred_term
MERGE (b:ESCO_Attitudes_Values {uri: row.level_2_uri})
ON CREATE SET b.title = row.level_2_preferred_term
MERGE (c:ESCO_Attitudes_Values {uri: row.level_3_uri})
ON CREATE SET c.title = row.level_3_preferred_term, c.description = row.description

MERGE (a)<-[:SUB_ELEMENT_OF]-(b)
MERGE (b)<-[:SUB_ELEMENT_OF]-(c)
 
{batch_size:1000}) YIELD * RETURN *; 

check_queries = []
MATCH p=(:ESCO_Skills)<-[:SUB_ELEMENT_OF]-(:ESCO_Skills)<-[:SUB_ELEMENT_OF]-(:ESCO_Skills) RETURN count(p) AS num 
MATCH p=(:ESCO_Knowledge)<-[:SUB_ELEMENT_OF]-(:ESCO_Knowledge)<-[:SUB_ELEMENT_OF]-(:ESCO_Knowledge) RETURN count(p) AS num 
MATCH (a:ESCO_Language) RETURN count(a) AS num 
MATCH p=(:ESCO_Attitudes_Values)<-[:SUB_ELEMENT_OF]-(:ESCO_Attitudes_Values)<-[:SUB_ELEMENT_OF]-(:ESCO_Attitudes_Values) RETURN count(p) AS num 
for cq in check_queries:
    results = run_reg_query(cq)['num'][0]
    if results <= 1:
        interrupt("ESCO KSATTs  "there should be more ESCO KSATTs or relationships between them")

num_skills =  MATCH (a:ESCO_Skills) RETURN count(a) AS num ['num'][0]
num_knowledge =  MATCH (a:ESCO_Knowledge) RETURN count(a) AS num ['num'][0]
num_language =  MATCH (a:ESCO_Language) RETURN count(a) AS num ['num'][0]
num_av =  MATCH (a:ESCO_Attitudes_Values) RETURN count(a) AS num ['num'][0]

In [ ]:
print("Add additional skills to the hierarchy")

 
 LOAD CSV FROM  '/usr/lib/memgraph/import/esco/trusted_transversalskillscollection_en.csv' WITH HEADER AS row RETURN row 
"
WHERE row.skillType = 'skill/competence' 
WITH split(row.broaderconceptpt, ' | ') AS broaderconcepttitles, split(row.broaderconcepturi, ' | ') AS broaderconcepturis, row
MERGE (a:ESCO_Skills {uri: broaderconcepturis[0]})
ON CREATE SET a.title = broaderconcepttitles[0]
MERGE (b:ESCO_Skills {uri: broaderconcepturis[1]})
ON CREATE SET b.title = broaderconcepttitles[1]
MERGE (c:ESCO_Skills {uri: row.conceptUri})
ON CREATE SET c.title = row.preferredlabel, c.alt_labels = row.altlabels, c.description = row.description

MERGE (a)<-[:SUB_ELEMENT_OF]-(c)
MERGE (b)<-[:SUB_ELEMENT_OF]-(c)
 {batch_size:1000}) YIELD * RETURN *; 

 
 LOAD CSV FROM  '/usr/lib/memgraph/import/esco/trusted_transversalskillscollection_en.csv' WITH HEADER AS row RETURN row 
"
WHERE row.skillType = 'knowledge' 
WITH split(row.broaderconceptpt, ' | ') AS broaderconcepttitles, split(row.broaderconcepturi, ' | ') AS broaderconcepturis, row
MERGE (a:ESCO_Knowledge {uri: broaderconcepturis[0]})
ON CREATE SET a.title = broaderconcepttitles[0]
MERGE (b:ESCO_Knowledge {uri: broaderconcepturis[1]})
ON CREATE SET b.title = broaderconcepttitles[1]
MERGE (c:ESCO_Knowledge {uri: row.conceptUri})
ON CREATE SET c.title = row.preferredlabel, c.alt_labels = row.altLabels, c.description = row.description

MERGE (a)<-[:SUB_ELEMENT_OF]-(c)
MERGE (b)<-[:SUB_ELEMENT_OF]-(c)

 {batch_size:1000}) YIELD * RETURN *; 

# prev_num_skills = num_skills
# num_skills = run_query("""MATCH (a:ESCO_Skills) RETURN count(a) AS num ['num'][0]
# if num_skills <= prev_num_skills:
#     interrupt("Additional ESCO Skills  "there should have been more ESCO Skills created")

# prev_num_knowledge = num_knowledge
# num_knowledge = run_query("""MATCH (a:ESCO_Knowledge) RETURN count(a) AS num ['num'][0]
# if num_knowledge <= prev_num_knowledge:
#     interrupt("Additional ESCO Knowledge  "there should have been more ESCO Skills created")

In [ ]:
print("Add languages to the hierarchy")

 
 LOAD CSV FROM  '/usr/lib/memgraph/import/esco/trusted_languageskillscollection_en.csv' WITH HEADER AS row RETURN row 
"
WHERE row.broaderconceptpt CONTAINS 'languages'
MATCH (a:ESCO_Language {title: 'languages'})
MERGE (b:ESCO_Language {uri: row.concepturi})
ON CREATE SET b.title = row.preferredlabel, b.alt_labels = row.altlabels, b.description = row.description

MERGE (a)<-[:SUB_ELEMENT_OF]-(c)
 {batch_size:1000}) YIELD * RETURN *; 

 
 LOAD CSV FROM  '/usr/lib/memgraph/import/esco/trusted_languageskillscollection_en.csv' WITH HEADER AS row RETURN row 
"
WHERE row.broaderconceptpt <> 'languages | language' AND row.broaderconceptpt <> 'language | languages'
MATCH (a:ESCO_Language {title: row.broaderconceptpt})
MERGE (b:ESCO_Language {uri: row.concepturi})
ON CREATE SET b.title = row.preferredlabel, b.alt_labels = row.altlabels, b.description = row.description

MERGE (a)<-[:SUB_ELEMENT_OF]-(b)
 {batch_size:1000}) YIELD * RETURN *; 

# check_queries = []
# MATCH p=(:ESCO_Language {title: 'languages'})<-[:SUB_ELEMENT_OF]-(:ESCO_Language) RETURN count(p) AS num 
# MATCH p=(:ESCO_Language)<-[:SUB_ELEMENT_OF]-(:ESCO_Language) RETURN count(p) AS num 
# for cq in check_queries:
#     results = run_query(cq)['num'][0]
#     if results <= 1:
#         interrupt("Additional ESCO KSATTs  "there should be more ESCO KSATTs or relationships between them")

# prev_num_language = num_language
# num_language = run_query("""MATCH (a:ESCO_Language) RETURN count(a) AS num ['num'][0]
# if num_language <= prev_num_language:
#     interrupt("Additional ESCO Languages  "there should have been more ESCO Languages created")

In [ ]:
print("Add additional skills and knowledge")
# TODO: These don't really connect to any other parts of the hierarchy as far as I could figure out.

# add other skills
 
 LOAD CSV FROM  '/usr/lib/memgraph/import/esco/trusted_skills_en.csv' WITH HEADER AS row RETURN row 
"
WHERE row.skilltype = 'skill/competence'
MERGE (a:ESCO_Skills {uri: row.concepturi})
ON CREATE SET a.title = row.preferredlabel, a.alt_title = row.altlabels, a.description = row.description
 {batch_size:1000, parallel:false}) YIELD * RETURN *; 

# add other knowledge
 
 LOAD CSV FROM  '/usr/lib/memgraph/import/esco/trusted_skills_en.csv' WITH HEADER AS row RETURN row 
"
WHERE row.skilltype = 'knowledge'
MERGE (a:ESCO_Knowledge {uri: row.concepturi})
ON CREATE SET a.title = row.preferredlabel, a.alt_title = row.altlabels, a.description = row.description
 {batch_size:1000, parallel:false}) YIELD * RETURN *; 

# prev_num_skills = num_skills
# num_skills = run_query("""MATCH (a:ESCO_Skills) RETURN count(a) AS num ['num'][0]
# if num_skills <= prev_num_skills:
#     interrupt("Additional ESCO Skills  "there should have been more ESCO Skills created")

# prev_num_knowledge = num_knowledge
# num_knowledge = run_query("""MATCH (a:ESCO_Knowledge) RETURN count(a) AS num ['num'][0]
# if num_knowledge <= prev_num_knowledge:
#     interrupt("Additional ESCO Knowledge  "there should have been more ESCO Skills created")

In [ ]:
print("Remove circular relationships AND add IDs based off of URIs")
esco_labels = ['ESCO_Skills', 'ESCO_Knowledge', 'ESCO_Language', 'ESCO_Attitudes_Values']
for el in esco_labels:
    MATCH (a:{})-[r:SUB_ELEMENT_OF]-(a) DELETE r RETURN count(r) AS num""".format(el))['num'][0]
    print("Deleted  results, "relationships.")
     
        MATCH (a:{})
        WITH split(a.uri, '/')[5] AS id_from_uri, a
        SET a.key = id_from_uri
    """.format(el))

In [ ]:
print("Create occupations and groups")
# create occupation groups - don't relate them yet
 
 LOAD CSV FROM  '/usr/lib/memgraph/import/esco/trusted_iscogroups_en.csv' WITH HEADER AS row RETURN row 
"
WITH row, split(row.concepturi, 'C')[1] AS code
MERGE (a:ESCO_Occupation_Group {uri: row.concepturi})
ON CREATE SET a.key = code, a.title = row.preferredlabel, a.alt_labels = row.altlabels, a.description = row.description
 {batch_size:1000}) YIELD * RETURN *; 

### WHAT THIS DOES ###
# find occupation groups where 
    # parent and child ids are not the same (e.g. 12, 121, 1213)
    # parent id is one less number in length than the child id (e.g. 12 is one number shorter than 121, same for 121 and 1213)
    # the parent id and a substring of the child id match
        # the substring of the child id starts at the first number in the id and is the length of the parent id
        # so in effect it's matching the substring of the child id that should be equal to the parent id 
        # e.g. 1213 child, 121 parent -> 1213 starts at first '1' and goes through '121' because that's the length of the parent 121
# create a relationship where the parent and child ids match up
 
    MATCH (a:ESCO_Occupation_Group), (b:ESCO_Occupation_Group)
    WHERE a.key <> b.key AND size(b.key)-1 = size(a.key) AND a.key = substring(b.key, 0, size(a.key))
    MERGE (a)<-[:SUB_ELEMENT_OF]-(b)
 

# create occupations and assign them to occupation groups
 
 LOAD CSV FROM  '/usr/lib/memgraph/import/esco/trusted_occupations_en.csv' WITH HEADER AS row RETURN row 
"
MATCH (a:ESCO_Occupation_Group {key: row.iscogroup})
MERGE (b:ESCO_Occupation {uri: row.concepturi})
SET b.title = row.preferredlabel, b.alt_labels = row.altlabels, b.description = row.description, b.key = row.code
MERGE (a)<-[:IN_GROUP]-(b)
 {batch_size:1000}) YIELD * RETURN *; 

 
    MATCH (a:ESCO_Occupation), (b:ESCO_Occupation)
    WHERE a.key <> b.key AND (size(b.key)-2 = size(a.key) OR size(b.key)-3 = size(a.key)) AND a.key = substring(b.key, 0, size(a.key))
    MERGE (a)<-[:SUB_ELEMENT_OF]-(b)
 

results1 =  MATCH p=(a:ESCO_Occupation_Group)<--(b:ESCO_Occupation_Group) WHERE a.key<>b.key AND size(b.key)-1 = size(a.key) AND b.key CONTAINS a.key RETURN count(p) AS num ['num'][0]
results2 =  MATCH p=(:ESCO_Occupation_Group)<--(:ESCO_Occupation_Group)<--(:ESCO_Occupation_Group)<--(:ESCO_Occupation_Group)<--(:ESCO_Occupation)<--(:ESCO_Occupation)<--(:ESCO_Occupation)<--(:ESCO_Occupation) RETURN count(p) AS num ['num'][0]
if results1 <= 1 or results2 <= 1:
    interrupt("ESCO Occupations and Groups  "there should be relationships")

MATCH (a:ESCO_Occupation) WHERE NOT (:ESCO_Occupation)<--(a) AND NOT (:ESCO_Occupation_Group)<--(a) RETURN count(a) AS num ['num'][0]
if results >= 1:
    interrupt("ESCO Occupations  "there should not be any ESCO Occupations not related to another occupation or group")

In [ ]:
#TODO: try relating skills and occupations using a bit of web scraping

# base_url = 'https://ec.europa.eu/esco/portal/occupation?uri=http%3A%2F%2Fdata.europa.eu%2Fesco%2Foccupation%2F00030d09-2b3a-4efd-87cc-c4ea39d27c34&conceptLanguage=en&full=true#&uri='
# uri_list = run_query("""MATCH (a:ESCO_Occupation) RETURN a.uri AS uri ['uri']
# for uri in uri_list:
#     try:
#         r = requests.get(base_url+uri)
#     except requests.exceptions.RequestException as e:
#         print('ERROR: could not access uri.')
#         raise SystemExit(e)
#     soup = BeautifulSoup(r.text, 'html.parser')

#     # https://stackoverflow.com/questions/5690686/using-nextsibling-from-beautifulsoup-outputs-nothing explains why you have to double up
#     header_list = ["Essential Knowledge"]
#     # header_list = ["Essential Knowledge  "Essential skills and competences  "Optional skills and competences  "Optional Knowledge"]
#     for h in header_list:
#         # clear uri list and decide on relationship and label type depending on header
#         ksatt_uris = []
#         if 'essential' in h.lower():
#             rel = 'ESSENTIAL_FOR'
#         else:
#             rel = 'OPTIONAL_FOR'
#         if 'knowledge' in h.lower():
#             label = 'ESCO_Knowledge'
#         else:
#             label = 'ESCO_Skills'
#         # look for that header
#         h2 = soup.find("h2  text=h)
#         # if the header actually exists
#         if h2 is not None:
#             ul = h2.nextSibling.nextSibling # find the tag after the header - will be the unordered list keeping all the other ksatt links underneath
#             for li in ul.find_all('li'): # find all the nested links
# a = li.find('a') # grab the individual pieces
# ksatt_uris.append(a.attrs['href']) # grab the link and add to our list of uris
#             # once all the uris are collected for a header, create relationships of the right type between the occupation and the KSATT with that URI! 
#             for ksatt_uri in ksatt_uris:
# run_query("""
#     MATCH (a:ESCO_Occupation {{uri: '{}'}})
#     MATCH (b:{} {{uri: '{}'}})
#     MERGE (a)<-[:{}]-(b)
#          "".format(uri, label, ksatt_uri, rel))


# # took 40m to get through 827/2942 ESCO_Occupations - essential AND optional knowledge/skills

# # took ___ to get through__________ ESCO_Occupations - essential knowledge only

# #28 per min avg
# # with 2942 = 100 min

## Upload NICE Data

NICE Framework Background  
- NIST (National Institute of Standards and Technology), NICCS (National Initiative for Cybersecurity Careers and Studies), OPM (Office of Personnel Management) all have a copy of the NICE (National Initiative for Cybersecurity Education) framework, which was developed by NICE. 
- Straight from OPM: OPM and DHS during the early stages of its collaborative endeavors co-led efforts to identify the cybersecurity workforce. With the direct engagement of over 20 Federal departments and agencies, and numerous public and private organizations, the National Initiative for Cybersecurity Education (NICE) developed the National Cybersecurity Workforce Framework (the Framework) to define cybersecurity work and lay a foundation for cybersecurity workforce efforts. The NICE Framework provides a common language and taxonomy, defines specialty areas and KSAs/competencies, and codifies talent.
- NICCS has NICE embedded in their website https://niccs.cisa.gov/workforce-development/cyber-security-workforce-framework 
- NIST has an excel spreadsheet https://www.nist.gov/itl/applied-cybersecurity/nice/nice-framework-resource-center/workforce-framework-cybersecurity-nice 
- OPM has it in a PDF https://www.opm.gov/policy-data-oversight/classification-qualifications/reference-materials/interpretive-guidance-for-cybersecurity-positions.pdf 
Other Resources  
  - https://csrc.nist.gov/projects/olir/focal-document-templates
  - https://nvlpubs.nist.gov/nistpubs/CSWP/NIST.CSWP.04162018.pdf
  - https://www.nist.gov/cyberframework/framework
  - https://niccs.cisa.gov/workforce-development/cyber-security-workforce-framework
  - Withdrawn: https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-181.pdf
  - Superseded by: https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-181r1.pdf
  - https://csrc.nist.gov/publications/detail/sp/800-181/rev-1/final
  - https://niccs.cisa.gov/workforce-development/cyber-career-pathways
  - https://niccs.cisa.gov/workforce-development
OPM Cybersecurity Codes are different from regular OPM Series Codes  
  - Grab Reference Spreadsheet https://www.nist.gov/itl/applied-cybersecurity/nice/nice-framework-resource-center/workforce-framework-cybersecurity-nice
  - Table of Contents has OPM codes, but these are not the same as the ones from OPM - they're 3 digits instead of the GS-XXXX structure. Researching into it, OPM actually has separate codes for these cybersecurity roles https://dw.opm.gov/datastandards/referenceData/2273/current?category=&q=cybersecurity. 
  - Tried to figure out more about these cybersecurity-specific codes
      - https://www.opm.gov/policy-data-oversight/classification-qualifications/classifying-general-schedule-positions/#url=Standards
      - made sense to find cybersecurity under 2200 IT Group so went digging into that PDF https://www.opm.gov/policy-data-oversight/classification-qualifications/classifying-general-schedule-positions/standards/2200/gs2200a.pdf
      - Searched for 2210 and found the IT Cybersecurity Specialist role, this additional document was linked https://www.opm.gov/policy-data-oversight/classification-qualifications/reference-materials/interpretive-guidance-for-cybersecurity-positions.pdf 
      - In that Interpretive Guidance for Cybersecurity Positions, there was the NICE framework and more explanations about these cybersecurity codes
      - Hard to tell if they are supposed to be under the 2210 focus, or are just completely separate. 
          - Don't think it's just 2210 because a section of the PDF includes that they also have overlap with the 0855, 0854, and 0391 series. 
PDW has Cybersecurity codes that match with the ones in the NICE Framework, but...  
  - These codes are actually results of something David did a couple years ago.
  - He compared position descriptions of employees with the descriptions for the cybersecurity codes and recommended the top 3 codes based on the doc2vec results. 
  - Main issues with this now:
      - Is it being maintained/updated live?
      - The position descriptions used at that time were more detailed than the ones we have now - would be comparing different results. 
  - Possible solution might be that we infer the cybersecurity code based on KSATT overlap, rather than code crosswalking. 

In [ ]:
print("Use table of contents to establish and relate NICE Categories, Areas and Workroles")

 
 LOAD CSV FROM  '/usr/lib/memgraph/import/nice/trusted_nice_areas_and_roles.csv' WITH HEADER AS row RETURN row 
"
MERGE (a:NICE_Category {title: toLower(row.nice_category_title), acronym: row.nice_category_acronym, description: toLower(row.nice_category_description)})
 {batch_size:10000}) YIELD * RETURN *;
 

 
 LOAD CSV FROM  '/usr/lib/memgraph/import/nice/trusted_nice_areas_and_roles.csv' WITH HEADER AS row RETURN row 
"
MERGE (a:NICE_Area {title: toLower(row.nice_specialty_area_title), acronym: row.nice_specialty_area_acronym, description: toLower(row.nice_specialty_area_description)})
 {batch_size:10000}) YIELD * RETURN *;
 

 
 LOAD CSV FROM  '/usr/lib/memgraph/import/nice/trusted_nice_areas_and_roles.csv' WITH HEADER AS row RETURN row 
"
WITH row, split(row.work_role_id, '-') AS work_role_id_components
MERGE (a:NICE_Workrole {key: row.work_role_id, key_1: work_role_id_components[0], key_2: work_role_id_components[1], key_3: work_role_id_components[2], title: toLower(row.work_role), description: toLower(row.work_role_description)})
WITH row, a
MATCH (b:NICE_Area {acronym: a.key_2})
MATCH (c:NICE_Category {acronym: a.key_1})
MERGE (a)-[:IN_AREA]->(b)
MERGE (b)-[:IN_CATEGORY]->(c)
 {batch_size:10000}) YIELD * RETURN *;
 

 
 LOAD CSV FROM  '/usr/lib/memgraph/import/nice/trusted_nice_areas_and_roles.csv' WITH HEADER AS row RETURN row 
"
MATCH (a:NICE_Workrole {key: row.work_role_id})
MERGE (b:OPM_Cybersecurity_Category {key: row.opm_code_fed_use})
MERGE (a)-[:IN_CATEGORY]->(b)
 {batch_size:10000}) YIELD * RETURN *;
 

MATCH a=(:NICE_Category)<-[:IN_CATEGORY]-(:NICE_Area)<-[:IN_AREA]-(:NICE_Workrole)-[:IN_CATEGORY]->(:OPM_Cybersecurity_Category) RETURN count(a) AS num ['num'][0]
if results == 0:
    interrupt("NICE Categories & Areas & Workroles  "there should be connected nodes")

In [ ]:
print("Add NICE KSATTs and relate them to NICE Workroles")
# add NICE_Tasks and connect to NICE_Workrole nodes
# don't add KSA id because there are KSAs with the same title but different IDs, and the IDs don't matter (no hierarchy or context)
 
 LOAD CSV FROM  '/usr/lib/memgraph/import/nice/trusted_master_ksatt_list_mapped.csv' WITH HEADER AS row RETURN row 
"
MATCH (a:NICE_Workrole {key: row.work_role_id})
MERGE (b:NICE_KSAs {title: toLower(row.title)})
ON CREATE SET b.key = row.id
MERGE (b)-[:FOUND_IN]->(a)
 {batch_size:10000}) YIELD * RETURN *;
 

# set NICE_KSAs labels to be more specific
 MATCH (a:NICE_KSAs) WHERE a.key CONTAINS 'K' SET a:NICE_Knowledge REMOVE a:NICE_KSAs, a.key 
 MATCH (a:NICE_KSAs) WHERE a.key CONTAINS 'S' SET a:NICE_Skills REMOVE a:NICE_KSAs, a.key 
 MATCH (a:NICE_KSAs) WHERE a.key CONTAINS 'A' SET a:NICE_Abilities REMOVE a:NICE_KSAs, a.key 
 MATCH (a:NICE_KSAs) WHERE a.key CONTAINS 'T' SET a:NICE_Tasks REMOVE a:NICE_KSAs, a.key 
check_queries = []
MATCH a=(:NICE_Knowledge)-[:FOUND_IN]->(:NICE_Workrole) RETURN count(a) AS num 
MATCH a=(:NICE_Skills)-[:FOUND_IN]->(:NICE_Workrole) RETURN count(a) AS num 
MATCH a=(:NICE_Abilities)-[:FOUND_IN]->(:NICE_Workrole) RETURN count(a) AS num 
MATCH a=(:NICE_Tasks)-[:FOUND_IN]->(:NICE_Workrole) RETURN count(a) AS num 
for cq in check_queries:
    results = run_reg_query(cq)['num'][0]
    if results == 0:
        interrupt("NICE Workroles & KSATTs  "there should be connected nodes")

## Crosswalk OPM and ONET

In [ ]:
print("Add in the OPM Series to ONET Occupation Crosswalk")

# create OPM Series nodes
 
 LOAD CSV FROM  '/usr/lib/memgraph/import/trusted_opm_onet_crosswalk.csv' WITH HEADER AS row RETURN row 
"
MERGE (:OPM_Series {key: row.opm_series_number, title: row.opm_series_title})
 {batch_size:1000}) YIELD * RETURN *; 

# create relationship between occupations and opm key's if they're in the right row
 
 LOAD CSV FROM  '/usr/lib/memgraph/import/trusted_opm_onet_crosswalk.csv' WITH HEADER AS row RETURN row 
"
MATCH (a:ONET_Occupation), (b:OPM_Series {key: row.opm_series_number})
WHERE a.key CONTAINS(row.`2010_soc_code`)
MERGE (a)-[r:IN_OPM {census_code: row.`2010_eeo_tabulation_census_code`, census_title: toLower(row.`2010_eeo_tabulation_census_occupation_title`)}]->(b)
 {batch_size:1000}) YIELD * RETURN *; 

result =  MATCH a=(:ONET_Occupation)-[:IN_OPM]->(:OPM_Series) RETURN count(a) AS num ['num'][0]
print(result)
if result == 0:
    interrupt("Occupations to OPM Series  "should be some relationships but there's none")

In [ ]:
print("Equate OPM Series to ONET Occupations for specific SOC Codes")

 MATCH (a:ONET_Occupation {key: '17-2071.00'}), (b:OPM_Series) WHERE b.key CONTAINS("855")
MERGE (a)-[:IN_OPM {census_code: '1410', census_title: toLower('ELECTRICAL & ELECTRONIC ENGINEERS')}]->(b); 

 MATCH (a:ONET_Occupation {key: '17-2072.00'}), (b:OPM_Series) WHERE b.key CONTAINS("855")
MERGE (a)-[:IN_OPM {census_code: '1410', census_title: toLower('ELECTRICAL & ELECTRONIC ENGINEERS')}]->(b); 

 MATCH (a:ONET_Occupation), (b:OPM_Series) WHERE a.key CONTAINS('17-206') AND b.key CONTAINS("854")
MERGE (a)-[:IN_OPM {census_code: '1400', census_title: toLower('COMPUTER HARDWARE ENGINEERS')}]->(b); 

# TODO: was 15-1111 but no SOC codes match that
# run_query("""MATCH (a:ONET_Occupation), (b:OPM_Series) WHERE a.key CONTAINS('15-1111') AND b.key CONTAINS("1550")
# MERGE (a)-[:IN_OPM {census_code: '1005', census_title: toLower('COMPUTER & INFORMATION RESEARCH SCIENTISTS')}]->(b); 
 MATCH (a:ONET_Occupation), (b:OPM_Series) WHERE a.key CONTAINS('15-1') AND b.key CONTAINS("1550")
MERGE (a)-[:IN_OPM {census_code: '1005', census_title: toLower('COMPUTER & INFORMATION RESEARCH SCIENTISTS')}]->(b); 

 MATCH (a:ONET_Occupation), (b:OPM_Series) WHERE a.key CONTAINS('15-1') AND b.key CONTAINS('2210')
MERGE (a)-[:IN_OPM {census_code: '1050', census_title: toLower('COMPUTER SUPPORT SPECIALISTS')}]->(b); 

In [ ]:
print("Check the OPM to ONET Crosswalk queries")

check_queries = []
MATCH p=(a:ONET_Occupation {key: '17-2071.00'})-[:IN_OPM]->(b:OPM_Series) WHERE b.key CONTAINS("855") RETURN count(p) AS num 
MATCH p=(a:ONET_Occupation {key: '17-2072.00'})-[:IN_OPM]->(b:OPM_Series) WHERE b.key CONTAINS("855") RETURN count(p) AS num 
MATCH p=(a:ONET_Occupation)-[:IN_OPM]->(b:OPM_Series) WHERE a.key CONTAINS('17-206') AND b.key CONTAINS("854") RETURN count(p) AS num 
# TODO: same here
# MATCH p=(a:ONET_Occupation)-[:IN_OPM]->(b:OPM_Series) WHERE a.key CONTAINS('15-1111') AND b.key CONTAINS("1550") RETURN count(p) AS num 
MATCH p=(a:ONET_Occupation)-[:IN_OPM]->(b:OPM_Series) WHERE a.key CONTAINS('15-1') AND b.key CONTAINS("1550") RETURN count(p) AS num 
MATCH p=(a:ONET_Occupation)-[:IN_OPM]->(b:OPM_Series) WHERE a.key CONTAINS('15-1') AND b.key CONTAINS("2210") RETURN count(p) AS num 

for cq in check_queries:
    results = run_reg_query(cq)['num'][0]
    if results <= 0:
        interrupt("ONET Occupation to OPM Crosswalk  "there should be specific relationships")